In [115]:
import pandas as pd 
import numpy as np
import re

In [116]:
# read excel file without footer and header information from the datafile
Energy = pd.read_excel('./datasets/Energy Indicators.xls', 
                       skiprows=16, 
                       skipfooter=283-245, 
                       usecols=[2, 3, 4, 5], 
                       na_values="...")
# rename the columns
Energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
# fill missing data with np.Nan
Energy.fillna(np.NaN)
Energy.head()

,Country,Energy Supply,Energy Supply per Capita,% Renewable
0,NaN,Petajoules,Gigajoules,%
1,Afghanistan,321,10,78.66928
2,Albania,102,35,100
3,Algeria,1959,51,0.55101
4,American Samoa,NaN,NaN,0.641026


In [117]:
# Convert Energy Supply to Gigajoules
# Energy['Energy Supply'] = Energy.loc[1:,[ 'Energy Supply' ]] * 1_000_000
Energy = Energy.drop([0])
Energy['Energy Supply'] = Energy['Energy Supply'] * 1_000_000

In [118]:
# Energy.loc[0, ['Energy Supply']] = "Gigajoules"

In [119]:
def extract_country_name(s):
    # Remove any parentheses and their contents
    s = re.sub(r'\(.*?\)', '', s)
    # Remove any trailing non-alphabetic characters
    s = re.sub(r'\W+$', '', s)
     # Remove any digits from the string
    s = re.sub(r'\d+', '', s)
    # Remove leading and trailing whitespace
    s = s.strip()
    return s

Energy['Country'] = Energy['Country'].apply(extract_country_name)

In [120]:
# Rename some countries
Energy.loc[Energy['Country'] == 'Republic of Korea', 'Country'] = 'South Korea'
Energy.loc[Energy['Country'] == 'United States of America', 'Country'] = 'United States'
Energy.loc[Energy['Country'] == 'United Kingdom of Great Britain and Northern Ireland', 'Country'] = 'United Kingdom'
Energy.loc[Energy['Country'] == 'China, Hong Kong Special Administrative Region', 'Country'] = 'Hong Kong'


In [121]:
Energy = Energy.reset_index(drop=True)
Energy.loc[Energy['Country'] == 'Iran']

,Country,Energy Supply,Energy Supply per Capita,% Renewable
98,Iran,9172000000,119,5.707721


In [122]:
# Load GDP data
GDP= pd.read_csv('./datasets/World Bank data.csv', skiprows=4)
GDP.head(3)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN
1,Andorra,AND,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,4.018196e+09,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN
2,Afghanistan,AFG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.030523e+10,1.172119e+10,1.214448e+10,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.029415e+10


In [123]:
GDP.loc[GDP['Country Name'] == 'Korea, Rep.', 'Country Name'] = 'South Korea'
GDP.loc[GDP['Country Name'] == 'Iran, Islamic Rep.', 'Country Name'] = 'Iran'
GDP.loc[GDP['Country Name'] == 'Hong Kong SAR, China', 'Country Name'] = 'Hong Kong'

In [124]:
GDP.loc[GDP['Country Name'] == 'Iran']

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
109,Iran,IRN,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,5.500820e+10,6.072406e+10,6.552689e+10,7.014134e+10,7.608598e+10,8.904776e+10,...,3.895523e+11,4.250646e+11,4.289909e+11,4.389208e+11,4.677902e+11,4.853309e+11,4.532569e+11,4.445926e+11,4.639027e+11,NaN


In [125]:
# Load journal datasets
ScimEn = pd.read_excel('./datasets/Scimagojr 3.xlsx')

In [126]:
ScimEn.loc[ScimEn['Country'] == 'Iran']

,Rank,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
12,13,Iran,8896,8819,57470,19125,6.46,72


In [127]:
GDP = GDP[[ GDP.columns[0] ] + GDP.columns[-10:].to_list()]
GDP.rename(columns={'Country Name': 'Country'}, inplace=True)
GDP.head(2)

,Country,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN
1,Andorra,4.018196e+09,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN


In [128]:
# ScimEn + Energy + GDP
df = pd.merge(ScimEn, Energy, on='Country', how='inner')

In [129]:
df = pd.merge(df, GDP, on='Country', how='inner')
df.set_index('Country', inplace=True)
df

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,127191000000,93,19.75491,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,90838000000,286,11.57098,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,18984000000,149,10.23282,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7920000000,124,10.60047,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,30709000000,214,17.28868,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Maldives,183,1,1,1,0,1.00,1,15000000,43,0,1.862196e+09,2.063143e+09,2.320557e+09,2.192661e+09,2.323402e+09,2.525755e+09,2.588810e+09,2.710479e+09,2.886134e+09,2.929826e+09
Belize,185,1,1,6,0,6.00,1,13000000,39,64.69003,1.285312e+09,1.299522e+09,1.341485e+09,1.352206e+09,1.397113e+09,1.426484e+09,1.479846e+09,1.499280e+09,1.560479e+09,1.590417e+09
Palau,186,1,1,0,0,0.00,0,3000000,152,18.75,2.074094e+08,2.072990e+08,1.957150e+08,1.779528e+08,1.838000e+08,1.930672e+08,1.992454e+08,1.943911e+08,2.026654e+08,2.216412e+08


In [130]:
res = df[df['Rank'] <= 15]
res

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,127191000000,93,19.75491,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,90838000000,286,11.57098,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,18984000000,149,10.23282,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7920000000,124,10.60047,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,30709000000,214,17.28868,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,10431000000,296,61.94543,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,13261000000,165,17.90153,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,33195000000,26,14.96908,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,10597000000,166,17.02028,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


In [131]:
res['Energy Supply'].mean()

26152866666.666668

In [132]:
gdp_only = res[res.columns[-10:].to_list()]

In [133]:
mean = gdp_only.mean(axis=1)
mean.sort_values(ascending=False)

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
dtype: float64

In [134]:
res['% Renewable'].max()
country = res.loc[res['% Renewable'] == res['% Renewable'].max()]



In [135]:
country.index[0]

'Brazil'

In [136]:
res.head()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,127191000000,93,19.75491,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,90838000000,286,11.57098,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,18984000000,149,10.23282,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7920000000,124,10.60047,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,30709000000,214,17.28868,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12


In [137]:
res['ratio'] = res['Self-citations'] / res['Citations']

/var/folders/qq/q7zb_56n3nd0hngl2qjjdt2c0000gn/T/ipykernel_88935/86064261.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['ratio'] = res['Self-citations'] / res['Citations']


In [138]:
max_ratio = res['ratio'].max()

In [139]:
res.loc[res['ratio'] == max_ratio].index[0]

'China'

In [140]:
res['Population'] = res['Energy Supply'] / res['Energy Supply per Capita']

/var/folders/qq/q7zb_56n3nd0hngl2qjjdt2c0000gn/T/ipykernel_88935/589594111.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['Population'] = res['Energy Supply'] / res['Energy Supply per Capita']


In [141]:
res.sort_values(by='Population', ascending=False).index[2]

'United States'

In [142]:
# res['Citable Documents per Person'] = res['Citable documents'] / res['Population']
# res['Citable Documents per Capita'] = res['Citable documents'] / res['Population']
res['Citable docs per Capita'] = res['Citable documents'] / res['Population']
res['Citable docs per Capita'].corr(res['Energy Supply per Capita'])

/var/folders/qq/q7zb_56n3nd0hngl2qjjdt2c0000gn/T/ipykernel_88935/3197404997.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['Citable docs per Capita'] = res['Citable documents'] / res['Population']


0.7940010435442946

In [143]:
type(res['Citable docs per Capita'])

pandas.core.series.Series

In [144]:
res['Citable docs per Capita'].corr(res['Energy Supply per Capita'])

0.7940010435442946

In [145]:
median_renewable = res['% Renewable'].median()
median_renewable

17.02028

In [146]:
def renewable_categories(country):
  if country >= median_renewable:
    return 1
  elif country < median_renewable:
    return 0

res.loc['China':, 'HighRenew'] = res['% Renewable'].apply(renewable_categories)

/var/folders/qq/q7zb_56n3nd0hngl2qjjdt2c0000gn/T/ipykernel_88935/620845672.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res.loc['China':, 'HighRenew'] = res['% Renewable'].apply(renewable_categories)


In [147]:
res.head(2)

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,...,2010,2011,2012,2013,2014,2015,ratio,Population,Citable docs per Capita,HighRenew
Country,,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.7,138,127191000000,93,19.75491,...,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12,0.689313,1367645161.290323,0.000093,1.0
United States,2,96661,94747,792274,265436,8.2,230,90838000000,286,11.57098,...,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13,0.335031,317615384.615385,0.000298,0.0


In [148]:
res['HighRenew'].sort_values(ascending=False)

Country
China                 1.0
Russian Federation    1.0
Canada                1.0
Germany               1.0
France                1.0
Italy                 1.0
Spain                 1.0
Brazil                1.0
United States         0.0
Japan                 0.0
United Kingdom        0.0
India                 0.0
South Korea           0.0
Iran                  0.0
Australia             0.0
Name: HighRenew, dtype: float64

In [149]:
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

In [150]:
res = res.reset_index()

In [151]:
def continent(x):
  for country in ContinentDict.keys():
    if x == country:
      return ContinentDict[country]

res['Continent'] = res['Country'].apply(continent)
res[[ 'Country', 'Continent' ]]

,Country,Continent
0,China,Asia
1,United States,North America
2,Japan,Asia
3,United Kingdom,Europe
4,Russian Federation,Europe
5,Canada,North America
6,Germany,Europe
7,India,Asia
8,France,Europe
9,South Korea,Asia


In [152]:
# then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.
res[[ 'Population', 'Continent' ]]

,Population,Continent
0,1367645161.290323,Asia
1,317615384.615385,North America
2,127409395.973154,Asia
3,63870967.741935,Europe
4,143500000.0,Europe
5,35239864.864865,North America
6,80369696.969697,Europe
7,1276730769.230769,Asia
8,63837349.39759,Europe
9,49805429.864253,Asia


In [153]:
res.pivot_table(values='Population', index='Continent',columns='Country', aggfunc=['size', 'sum', 'mean', 'std']).stack()

/var/folders/qq/q7zb_56n3nd0hngl2qjjdt2c0000gn/T/ipykernel_88935/2322635628.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  res.pivot_table(values='Population', index='Continent',columns='Country', aggfunc=['size', 'sum', 'mean', 'std']).stack()


size                sum               mean
Continent     Country                                                       
Asia          China                1.0  1367645161.290323  1367645161.290323
              India                1.0  1276730769.230769  1276730769.230769
              Iran                 1.0    77075630.252101    77075630.252101
              Japan                1.0   127409395.973154   127409395.973154
              South Korea          1.0    49805429.864253    49805429.864253
Australia     Australia            1.0    23316017.316017    23316017.316017
Europe        France               1.0     63837349.39759     63837349.39759
              Germany              1.0    80369696.969697    80369696.969697
              Italy                1.0    59908256.880734    59908256.880734
              Russian Federation   1.0        143500000.0        143500000.0
              Spain                1.0    46443396.226415    46443396.226415
              United Kingdom       1.0    63870967.741935    63870967.741935
North America Canada               1.0    35239864.864865    35239864.864865
              United States        1.0   317615384.615385   317615384.615385
South America Brazil               1.0   205915254.237288   205915254.237288

In [154]:
res.groupby('Continent').agg({'Population': ['size', 'sum', 'mean', 'std']})

Population                                                  
                    size               sum              mean           std
Continent                                                                 
Asia                   5   2898666386.6106   579733277.32212  6.790979e+08
Australia              1   23316017.316017   23316017.316017           NaN
Europe                 6  457929667.216372   76321611.202729  3.464767e+07
North America          2   352855249.48025  176427624.740125  1.996696e+08
South America          1  205915254.237288  205915254.237288           NaN

In [157]:
q_13_df = res

In [ ]:
q_13_df['% Renewable'] = pd.cut(q_13_df['% Renewable'], 5)
q_13_df = q_13_df.set_index(['Continent', '% Renewable'])

In [167]:
q_12 = q_13_df.groupby(level=(0, 1), observed=False).size()


In [173]:
len(q_12)
q_12

Continent      % Renewable     
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    0
Australia      (2.212, 15.753]     1
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    0
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
               (42.701, 56.174]    0
               (56.174, 69.648]    0
North America  (2.212, 15.753]     1
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    1
South America  (2.212, 15.753]     0
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    1
dtype: int64

In [174]:
res

,Country,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,...,2011,2012,2013,2014,2015,ratio,Population,Citable docs per Capita,HighRenew,Continent
0,China,1,127050,126767,597237,411683,4.70,138,127191000000,93,...,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12,0.689313,1367645161.290323,0.000093,1.0,Asia
1,United States,2,96661,94747,792274,265436,8.20,230,90838000000,286,...,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13,0.335031,317615384.615385,0.000298,0.0,North America
2,Japan,3,30504,30287,223024,61554,7.31,134,18984000000,149,...,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12,0.275997,127409395.973154,0.000238,0.0,Asia
3,United Kingdom,4,20944,20357,206091,37874,9.84,139,7920000000,124,...,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12,0.183773,63870967.741935,0.000319,0.0,Europe
4,Russian Federation,5,18534,18301,34266,12422,1.85,57,30709000000,214,...,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12,0.362517,143500000.0,0.000128,1.0,Europe
5,Canada,6,17899,17620,215003,40930,12.01,149,10431000000,296,...,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12,0.190369,35239864.864865,0.0005,1.0,North America
6,Germany,7,17027,16831,140566,27426,8.26,126,13261000000,165,...,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12,0.195111,80369696.969697,0.000209,1.0,Europe
7,India,8,15005,14841,128763,37209,8.58,115,33195000000,26,...,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12,0.288973,1276730769.230769,0.000012,0.0,Asia
8,France,9,13153,12973,130632,28601,9.93,114,10597000000,166,...,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12,0.218943,63837349.39759,0.000203,1.0,Europe
9,South Korea,10,11983,11923,114675,22595,9.57,104,11007000000,221,...,1.134796e+12,1.160809e+12,1.194429e+12,1.234340e+12,1.266580e+12,0.197035,49805429.864253,0.000239,0.0,Asia
